In [29]:
# import

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 找不到element的例外
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

In [3]:
website = 'website URL '
accont = 'your accont'
password = 'your password'
destDir = 'your destnition to download files'

with open('.credential/website') as wb:
    website = wb.readline()

with open('.credential/accont') as ac:
    accont = ac.readline()

with open('.credential/password') as pw:
    password = pw.readline()

with open('.credential/destDir') as dd:
    destDir = dd.readline()

In [4]:
# 啟動瀏覽器工具的選項
my_options = webdriver.EdgeOptions()
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

In [5]:
# Global Variable

driver = False
courses_Num = 0

In [35]:
# functions for driver

def init():
    global driver
    if not driver:
        driver = webdriver.Edge(options=my_options)

# visit a website
def visit(URL):
    driver.get(URL)
    print('Going to', website)

# wait till some element shows up. Success with True, failed with False.
def wait_Element(CSS_target):
    try:
        WebDriverWait(driver, 4).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, CSS_target)
            )
        )
        return True

    except TimeoutException:
        print('ERROR: Time Out!')
        return False

def login():
    # 等登入按鈕出現
    # 若先前已登入則返回
    if not wait_Element('button.btn.btn-primary.btn-lg.btn-block'):
        print('Failed: Aleady logged in!')
        return
    # 輸入帳密並登入
    driver.find_element(By.CSS_SELECTOR, 'input[placeholder=\"帳號\"]').send_keys(accont)
    driver.find_element(By.CSS_SELECTOR, 'input[placeholder=\"密碼 (區分大小寫)\"]').send_keys(password)
    driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary.btn-lg.btn-block').click()
    # 等詢問視窗pop up，點選保持登入
    wait_Element('.btn.btn-default.keepLoginBtn')
    driver.find_element(By.CSS_SELECTOR, '.btn.btn-default.keepLoginBtn').click()

def get_Courses_Num():
    global courses_Num
    wait_Element('tbody')
    courses_Num = len(driver.find_elements(By.CSS_SELECTOR, 'tbody tr'))
    print(courses_Num, 'courses found.')

# download material file in the material page
def download_Material():
    global driver
    if wait_Element('a[title=\"下載\"]'):
        driver.find_element(By.CSS_SELECTOR, 'a[title=\"下載\"]').click()

# traverse materials in a course's materials page
def visit_Materials():
    wait_Element('tbody')

    # How many materials are in the course
    materials_Num = len(driver.find_elements(By.CSS_SELECTOR, 'tbody tr'))
    
    # If the course has no material
    try:
        driver.find_element(By.CSS_SELECTOR, 'tr#noData')
        return
    except NoSuchElementException:
        pass

    print(materials_Num)
    for i in range(1, materials_Num+1):
        # WebElement of ith material link
        WE_material_link = driver.find_element(By.CSS_SELECTOR, f'tbody tr:nth-child({i}) :nth-child(2) [href]')

        # Name and Link of ith material
        name_Material = WE_material_link.get_attribute('title')
        link_Material = WE_material_link.get_attribute('href')
        driver.get(link_Material)
        download_Material()
        sleep(1)
        driver.back()

# traverse courses' materials page of a student
def visit_Courses():
    global courses_Num
    global driver
    for i in range(1, 2):
    #for i in range(1, courses_Num+1):
        wait_Element('tbody')

        # WebElement of ith course
        WE_course = driver.find_element(By.CSS_SELECTOR, f'tbody tr:nth-child({i})')

        # Semester of ith course. Number as string.
        semester_Course = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(1) .text-overflow').text

        # Name of ith course
        name_Course = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(3) [href]').get_attribute('title')

        # Link to ith course material 
        link_material = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(3) [href]').get_attribute('href')
        link_material = link_material[0:-5] + 'material' + link_material[-6:]

        driver.get(link_material)
        visit_Materials()
        driver.back()


In [36]:
# main

if __name__ == '__main__':
    init()
    visit(website)
    login()
    get_Courses_Num()
    visit_Courses()


Going to https://ncueeclass.ncu.edu.tw/dashboard/historyCourse
ERROR: Time Out!
Failed: Aleady logged in!
34 courses found.
15


KeyboardInterrupt: 